In [ ]:
!pip install --upgrade gensim -q
!pip install matplotlib -q

In [ ]:
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import spacy
import string

In [ ]:
gensim.__version__

'4.3.2'

In [ ]:
np.random.seed(42) #everytime the random no generated is same

importing api by gensim and models available to train

In [ ]:
import gensim.downloader as api
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


it means 50dimensional glove vectors containg twitter data, here I can take any of the above models but I took this because it would take **less time** as compared to models with 300 dimensions from the above list

In [ ]:
wv = api.load('glove-twitter-50')

[==================================================] 100.0% 199.5/199.5MB downloaded


In [ ]:
type(wv)

In [ ]:
wv['adarsh']  #here if we pass anything to wv it would return its vector

In [ ]:
len(wv['adarsh'])

50

In [ ]:
wv.similarity("apple", "adarsh")

0.039472006

In [ ]:
wv.similarity("apple", "human")

0.4398617

now here just keep in mind that if we'll use large dimensions vector then the similarity points would be much higher and meaningful ,

In [ ]:
pairs = [
    ('adarsh', 'human'),   # obviously I'm a human
    ('adarsh', 'alien'),   # not an alien
    ('adarsh', 'apple'),  # how can a human be an apple
    ('adarsh', 'coder'),    # yes I'm a coder
    ('adarsh', 'moon'),  #least meaningful
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

'adarsh'	'human'	0.18
'adarsh'	'alien'	0.08
'adarsh'	'apple'	0.04
'adarsh'	'coder'	0.11
'adarsh'	'moon'	-0.09


In [ ]:
print(wv.most_similar(positive=['adarsh', 'coder'], topn=5)) #top 5 words similar to these words

[('lawfirm', 0.789028525352478), ('ediscovery', 0.7544506192207336)]


In [ ]:
print(wv.doesnt_match(['employee', 'human', 'man', 'apple', 'coder', 'ceo'])) #print the odd one out, *keep in mind if you arent getting satisfying ans bc your model is being trained on lesser dimensions*

coder


In [ ]:
wv.most_similar(positive=['human', 'alien'], negative=['coder'], topn=3)

[('earth', 0.7817738056182861),
 ('planet', 0.744184136390686),
 ('world', 0.7263190150260925)]

In [ ]:
wv.most_similar(positive=['human', 'alien'], topn=3)

[('planet', 0.8321497440338135),
 ('species', 0.8213618993759155),
 ('killer', 0.816745936870575)]

Using PCA(Principle component Analysis) to convert **3D->2D visualization**

In [ ]:
words = ["coder",'mars','kid','earth','man','woman']

In [ ]:
sample_vectors = np.array([wv[word] for word in words])
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
result = pca.fit_transform(sample_vectors)
result

array([[ 4.893806  , -1.1585567 ],
       [ 0.42602214,  2.9140806 ],
       [-0.96796274, -0.8983191 ],
       [-0.7051693 ,  1.8608824 ],
       [-2.5362284 , -1.2994022 ],
       [-1.1104674 , -1.418685  ]], dtype=float32)

Now I'm plotting these words to see pictorial representation

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(result[:,0], result[:,1])
for i, word in enumerate(words):
    plt.annotate(word, xy=(result[i, 0], result[i, 1]))
plt.show()

wohho, now from the 2D rep. we can see man,woman,kid are closely related as compared to earth,mars etc

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Calculating **toxicity based on 12 different factors**.

In [12]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/Toxiccommentscsv.csv",error_bad_lines=False, engine="python")
data.head()

<ipython-input-12-f78dbc53511c>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv("/content/drive/MyDrive/Toxiccommentscsv.csv",error_bad_lines=False, engine="python")


,CommentId,VideoId,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism
0,Ugg2KwwX0V8-aXgCoAEC,04kJtp6pVXI,If only people would just take a step back and...,False,False,False,False,False,False,False,False,False,False,False,False
1,Ugg2s5AzSPioEXgCoAEC,04kJtp6pVXI,Law enforcement is not trained to shoot to app...,True,True,False,False,False,False,False,False,False,False,False,False
2,Ugg3dWTOxryFfHgCoAEC,04kJtp6pVXI,\nDont you reckon them 'black lives matter' ba...,True,True,False,False,True,False,False,False,False,False,False,False
3,Ugg7Gd006w1MPngCoAEC,04kJtp6pVXI,There are a very large number of people who do...,False,False,False,False,False,False,False,False,False,False,False,False
4,Ugg8FfTbbNF8IngCoAEC,04kJtp6pVXI,"The Arab dude is absolutely right, he should h...",False,False,False,False,False,False,False,False,False,False,False,False


In [3]:
def sent_vec(sent):
    vector_size = wv.vector_size
    wv_res = np.zeros(vector_size)
    # print(wv_res)
    ctr = 0
    for w in sent:
        if w in wv:
            ctr += 1
            wv_res += wv[w]
    wv_res = wv_res/ctr                  #vector rep. of each word=(all vectors associated with each word in that sentence/no. of words)
    return wv_res

In [2]:
import gensim.downloader as api
wv = api.load('glove-twitter-50')

[==================================================] 100.0% 199.5/199.5MB downloaded


In [5]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


Fn for **tokenization and lemmatization**

In [6]:
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_lg") # You can replace "en_core_web_sm" with the appropriate model for your language

# Define your stop_words and punctuations if they are not defined
stop_words = set(["your", "stop", "words"])
punctuations = set(["your", "punctuations"])

def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    doc = nlp(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [word.lemma_.lower().strip() for word in doc]

    # Removing stop words
    mytokens = [word for word in mytokens if word not in stop_words and word not in punctuations]

    # Return preprocessed list of tokens
    return mytokens


In [8]:
import numpy as np


In [9]:
sentence = "are you human"
tokens = spacy_tokenizer(sentence)
embedding_result = sent_vec(tokens)
print(embedding_result)

[-0.059465    0.24890667 -0.28140399  0.18544199 -0.06903234  0.10880933
  0.83542669  0.20379333 -0.22133566  0.71731665 -0.16827666 -0.165102
 -5.33030009 -0.2076     -0.04737666 -0.21287201  0.15255332 -0.19123333
  0.01480333 -0.15755166  0.26792666  0.23550333 -0.47870002  0.56845667
 -0.12374767  0.67854332 -0.01673667 -0.09532817  0.642589    0.20044001
 -0.06262332 -0.12818733  0.06609     0.06821933  0.41985668  0.10867001
 -0.06085999  0.09153667 -0.22903834 -0.13608667 -0.74343334 -0.34762666
  0.42811333 -0.10498667  0.327719   -0.408569    0.25737333  0.28776234
 -0.14426    -0.26124667]


In [10]:
nlp = spacy.load("en_core_web_sm")  #noted all stop words in our file
stop_words = nlp.Defaults.stop_words
print(stop_words)

{'yourself', 'five', 'very', "'s", 'n’t', 'already', 'others', 'you', 'becomes', 'nothing', 'per', 'their', 'mine', 'go', 'in', 'front', 'cannot', 'then', 'thence', 'but', '’s', 'above', 'whatever', 'neither', 'should', 'those', 'this', 'together', 'another', 'its', 'rather', 'third', 'herself', "'m", 'some', 'give', 'due', 'sometimes', 'was', 'even', 'put', 'meanwhile', 'us', 'about', 'almost', 'namely', 'while', 'these', 'any', 'his', 'doing', 'ever', 'really', '’m', 'eleven', 'six', "'ll", "'re", 'whereas', 'only', 'seeming', 'himself', "n't", 'each', 'yours', 'could', 'regarding', 'off', 'within', 'he', '‘d', '’re', 'least', 'unless', 'take', 'since', 'name', '’d', 'first', 'it', 'wherever', 'either', 'itself', 'using', 'n‘t', 'enough', 'every', 'whereby', 'than', 'me', 'amongst', 'none', 'is', 'by', 'for', 'why', 'back', 'had', 'call', 'throughout', 'are', 'indeed', 'towards', 'thereafter', 'see', 'often', 'below', 'whoever', 'besides', 'will', 'down', 'therein', 'last', 'when', '

In [11]:
import string
punctuations = string.punctuation #we have all the punctuations in string module of python
print(punctuations)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


making a **new column named "Newtokens" and converting text into list of tokens**

In [13]:
import pandas as pd

In [15]:
data = pd.read_csv('/content/drive/MyDrive/Toxiccommentscsv.csv')

In [16]:
data['Newtokens'] = data['Text'].apply(spacy_tokenizer)


In [17]:
data.head()

,CommentId,VideoId,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism,Newtokens
0,Ugg2KwwX0V8-aXgCoAEC,04kJtp6pVXI,If only people would just take a step back and...,False,False,False,False,False,False,False,False,False,False,False,False,"[people, step, case, people, situation, lump, ..."
1,Ugg2s5AzSPioEXgCoAEC,04kJtp6pVXI,Law enforcement is not trained to shoot to app...,True,True,False,False,False,False,False,False,False,False,False,False,"[law, enforcement, train, shoot, apprehend, tr..."
2,Ugg3dWTOxryFfHgCoAEC,04kJtp6pVXI,\nDont you reckon them 'black lives matter' ba...,True,True,False,False,True,False,False,False,False,False,False,False,"[reckon, black, life, matter, banner, hold, wh..."
3,Ugg7Gd006w1MPngCoAEC,04kJtp6pVXI,There are a very large number of people who do...,False,False,False,False,False,False,False,False,False,False,False,False,"[large, number, people, like, police, officer,..."
4,Ugg8FfTbbNF8IngCoAEC,04kJtp6pVXI,"The Arab dude is absolutely right, he should h...",False,False,False,False,False,False,False,False,False,False,False,False,"[arab, dude, absolutely, right, shoot, 6, extr..."


In [18]:
data['vec'] = data['Newtokens'].apply(sent_vec)

In [19]:
data.head()

,CommentId,VideoId,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism,Newtokens,vec
0,Ugg2KwwX0V8-aXgCoAEC,04kJtp6pVXI,If only people would just take a step back and...,False,False,False,False,False,False,False,False,False,False,False,False,"[people, step, case, people, situation, lump, ...","[0.287637183993494, 0.39703066171877965, 0.003..."
1,Ugg2s5AzSPioEXgCoAEC,04kJtp6pVXI,Law enforcement is not trained to shoot to app...,True,True,False,False,False,False,False,False,False,False,False,False,"[law, enforcement, train, shoot, apprehend, tr...","[0.07915546214924409, 0.49968385008665234, -0...."
2,Ugg3dWTOxryFfHgCoAEC,04kJtp6pVXI,\nDont you reckon them 'black lives matter' ba...,True,True,False,False,True,False,False,False,False,False,False,False,"[reckon, black, life, matter, banner, hold, wh...","[0.15751607860875083, -0.07351378630846739, 0...."
3,Ugg7Gd006w1MPngCoAEC,04kJtp6pVXI,There are a very large number of people who do...,False,False,False,False,False,False,False,False,False,False,False,False,"[large, number, people, like, police, officer,...","[0.35358490085317973, 0.4757189072579855, -0.2..."
4,Ugg8FfTbbNF8IngCoAEC,04kJtp6pVXI,"The Arab dude is absolutely right, he should h...",False,False,False,False,False,False,False,False,False,False,False,False,"[arab, dude, absolutely, right, shoot, 6, extr...","[0.15596105119115428, 0.5661786800544513, 0.31..."


In [21]:
X = data['vec'].to_list()
y = data['IsToxic'].to_list()

In [25]:
X[13]

array([-0.50266202,  0.00901013,  0.028302  ,  0.15687601,  0.06172801,
        0.56048801,  0.55072001, -0.081192  ,  0.08259199,  0.2880106 ,
       -0.01384167,  0.14456401, -3.49548001, -0.162361  ,  0.20931401,
        0.3569988 , -0.12908884, -0.0861434 ,  0.27482601, -0.3120966 ,
       -0.38496401,  0.3571674 ,  0.14216201,  0.40782399,  0.2727782 ,
        0.174     , -0.17922801,  0.8543758 ,  0.02153199,  0.17254481,
        0.07429921, -0.16479   ,  0.2913286 , -0.105238  ,  0.1872996 ,
        0.06277   , -0.09875201,  0.2598648 , -0.024166  ,  0.35408401,
       -0.30596998,  0.0442798 ,  0.20919759, -0.2407716 , -0.02779402,
       -0.09910359,  0.440718  ,  0.04194999,  0.505002  , -0.1159774 ])

In [26]:
y[13]

True

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y)

Testing data using **Supervised Learning algo i.e. SVM(SUPPORT VECTOR MACHINE)**

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline

# Assuming X_train is your feature matrix with missing values
# Create a pipeline with an imputer and the SVM classifier
clf = make_pipeline(SimpleImputer(strategy='mean'), SVC())

# Fit the pipeline to your data
clf.fit(X_train, y_train)



In [48]:
predicted = clf.predict(X_test)

# Print evaluation metrics
print("SVM Accuracy:", metrics.accuracy_score(y_test, predicted))
print("SVM Precision:", metrics.precision_score(y_test, predicted))
print("SVM Recall:", metrics.recall_score(y_test, predicted))

SVM Accuracy: 0.725
SVM Precision: 0.6989247311827957
SVM Recall: 0.7065217391304348


Testing data using **Supervised Learning algo i.e. LOGISTIC REGRESSION**

In [49]:
from sklearn.linear_model import LogisticRegression
clf = make_pipeline(SimpleImputer(strategy='mean'), LogisticRegression())


In [50]:
clf.fit(X_train, y_train)

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('logisticregression', LogisticRegression())])

In [51]:
predicted2 = clf.predict(X_test)

In [52]:
from sklearn import metrics
predicted = clf.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted2))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted2))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted2))

Logistic Regression Accuracy: 0.71
Logistic Regression Precision: 0.7073170731707317
Logistic Regression Recall: 0.6304347826086957
